# RECONOCIMIENTO DE TONOS

Primera aproximación: Hallar la nota más presente, máximo de la fft, en cada ventana.

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from scipy.fftpack import fft
from scipy.io import wavfile # get the api
import os
import seaborn as sns
# From V8
import pandas as pd
from scipy.signal import find_peaks,correlate
from sklearn.cluster import KMeans

In [2]:
# Inputs
PATH = 'C:\\Users\\Javier\\Desktop\\TFG\\ReconocimientoDeTonos'      # Path of project's directory
AUDIO_FILE = 'DoM-piano.wav'                                    # Audio file's name
WINDOW_SIZE_SECS = 0.2                                               # Size of the fft window in seconds
OVERLAPPING_SECS = 0.15                                              # Window's overlapping in seconds
INTENSITY_THRESHOLD = 0.001                                          # Intensity (relevance) threshold for frequencies

In [3]:
# Global variables and files' statistics
NOTES = ["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]       # The twelve notes' names

SAMPLE_RATE, data = wavfile.read(os.path.join(PATH,AUDIO_FILE))      # Get sample rate (samples per second) and signal data
signal = data if data.ndim == 1 else data.T[0]                       # Get the first channel
WINDOW_SIZE_SAMPLES = int(SAMPLE_RATE * WINDOW_SIZE_SECS)            # Size of the fft window in samples
OVERLAPPING_SAMPLES = int(SAMPLE_RATE * OVERLAPPING_SECS)            # Size of overlapping in samples
AUDIO_SIZE_SECS = len(signal) / SAMPLE_RATE                          # Size of the audio file in seconds

print("Sample rate: " + str(SAMPLE_RATE) + " Hz")                   
print("Signal: " + str(signal))                                      
print("Window size: " + str(WINDOW_SIZE_SECS) + " s = " + str(WINDOW_SIZE_SAMPLES) + " samples")
print("Overlapping: " + str(OVERLAPPING_SECS) + " s = " + str(OVERLAPPING_SAMPLES) + " samples")
print("Audio length: " + str(AUDIO_SIZE_SECS) + " s")

Sample rate: 44100 Hz
Signal: [0.01104736 0.01083374 0.01083374 ... 0.         0.         0.        ]
Window size: 0.2 s = 8820 samples
Overlapping: 0.15 s = 6615 samples
Audio length: 13.933469387755101 s


<ipython-input-3-7c3418ec4c51>:4: WavFileWarning: Chunk (non-data) not understood, skipping it.
  SAMPLE_RATE, data = wavfile.read(os.path.join(PATH,AUDIO_FILE))      # Get sample rate (samples per second) and signal data


In [26]:
# Functions 1
def graph2D(x,y,file):
    rcParams['font.family'] = 'Book Antiqua'
    fig,ax = plt.subplots()
    ax.plot(x,y,label='Módulo de la FFT',color='black')
    ax.set_title('Densidad espectral de potencia')
    ax.set_xlabel('Frecuencias')
    ax.set_ylabel('Intensidad')
    ax.grid(True);
    fig.savefig(file)
    plt.close(fig)
    
def graph2D_old(X,Y,file):
    plt.figure()
    sns.set_style('darkgrid')
    sns.lineplot(x=X,y=Y,color='black')
    plt.savefig(file)
    plt.close()

def freq_to_number(f):                                                      # Transforms any note's frequency into its midi number 
    return 69 + 12*np.log2(f/440.0)    

def number_to_freq(n):                                                      # Transforms any note's midi number into its frequency
    return 440 * 2.0**((n-69)/12.0)

def note_name(n):                                                           # Gets the note's name given its midi number
    return NOTES[n % 12] + str(int(n/12 - 1))

def extract_window(audio, window_number):                                   # Returns samples of window number <window-number> and true or false whether it's the last window 
    begin = window_number * (WINDOW_SIZE_SAMPLES - OVERLAPPING_SAMPLES)
    end = begin + WINDOW_SIZE_SAMPLES
    
    if end < len(signal): # Commonly
        return False, audio[begin:end]
    else: # The window surpasses the audio data => Complete last elements of the window with zeros
        return True, np.concatenate([audio[begin:len(signal)-1],np.zeros(end-len(signal)+1,dtype=float)])

def detect_note_1(fft):                                                       # Returns the most suitable note for a given fft
    freqs = np.fft.rfftfreq(WINDOW_SIZE_SAMPLES, 1/SAMPLE_RATE) # The array of frequencies to evaluate in the fft
    F = np.abs(fft.real) # Evaluations of those frequencies
    
    if np.max(F) < INTENSITY_THRESHOLD: # If there are not high enough maximums, we cannot know
        return "Unknown"
    else:
        n = freq_to_number(freqs[np.argmax(F)]) # Midi number of absolute maximum's frequency 
        n_approx = int(round(n)) # Round the midi number to fit a note
        return note_name(n_approx)

In [5]:
# Main 1
def main_1():
    hanning = 0.5 * (1 - np.cos(np.linspace(0,2*np.pi,WINDOW_SIZE_SAMPLES,False)))  # The hanning window function

    notes = []
    window_number = 0
    last_window = False
    while not(last_window):
        last_window, window = extract_window(signal, window_number)
        window_number += 1

        fft = np.fft.rfft(window * hanning)
        notes.append(detect_note_1(fft))
        
    print(notes)

In [6]:
main_1()

['C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C5', 'C4', 'C4', 'C4', 'C5', 'C5', 'C5', 'C4', 'C5', 'G5', 'D4', 'D4', 'D5', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D5', 'D4', 'D4', 'D5', 'D4', 'F4', 'E6', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'B1', 'D#0', 'F4', 'F4', 'F4', 'F4', 'F4', 'F5', 'F5', 'F5', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'G#4', 'D6', 'G4', 'G5', 'G4', 'G5', 'G4', 'G5', 'G4', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A5', 'D#0', 'D#0', 'D#0', 'A1', 'D#0', 'D#0', 'B4', 'B4', 'B4', 'B4', 'B5', 'B4', 'B4', 'B4', 'B4', 'F#6', 'B4', 'B4', 'B5', 'B4', 'D#0', 'D#0', 'D#0', 'D#0', 'D#0', 'B4', 'C5', 'C5', 'C5', 'C5', 'C6', 'C5', 'C6', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B5', 'B4', 'A1', 'B4', 'B4', 'A1', 'D#0', 'D#0', 'A4', 'A4', 'A4', 'A4', 'A5', 

Segunda aproximación: Hallar el primer máximo de la fft en cada ventana, entre los puntos cuya evaluación se encuentre por encima de la media de las evaluaciones.

In [6]:
# Functions 2
def graph2D_T_old(X,Y,threshold,file):
    plt.figure()
    sns.set_style('darkgrid')
    sns.lineplot(x=X,y=Y,color='g')
    sns.lineplot(x=X,y=np.full(len(X),threshold),color='r')
    plt.savefig(file)
    plt.close()

def graph2D_T(x,y,threshold,file):
    rcParams['font.family'] = 'Book Antiqua'
    fig,ax = plt.subplots()
    ax.plot(x,y,label='Módulo de la FFT',color='black')
    ax.plot(x,np.full(len(y),threshold),label='Umbral',color='red')
    ax.set_title('Módulo de la FFT')
    ax.set_xlabel('Frecuencias')
    ax.set_ylabel('Intensidad')
    ax.grid(True);
    fig.savefig(file)
    plt.close(fig)
    
def first_maximum(freqs,F):                                                   # Returns frequency of first maximum
    max_threshold = (np.max(F) + np.min(F))/2 # Threshold for the max
    maximums = []
    
    for i in range(1,len(F)):
        if F[i] > max_threshold and ((i == 0 and F[i+1] < F[i]) or (0 < i and i < len(F) and F[i-1] < F[i] and F[i+1] < F[i]) or (i == len(F)-1 and F[i-1] < F[i])):
            maximums.append(i)
    
    return freqs[min(maximums)]
    
def detect_note_2(fft,wnum):                                                       # Returns the most suitable note for a given fft
    freqs = np.fft.rfftfreq(WINDOW_SIZE_SAMPLES, 1/SAMPLE_RATE) # The array of frequencies to evaluate in the fft
    F = np.abs(fft.real) # Evaluations of those frequencies
    graph2D_T(freqs[0:1000],F[0:1000],(np.max(F) + np.min(F))/2,"Plots\\w" + str(wnum) + "_V2.pdf")
    
    if np.max(F) < INTENSITY_THRESHOLD: # If there are not high enough maximums, we cannot know
        return "Unknown"
    else:
        n = freq_to_number(first_maximum(freqs,F)) # Midi number of absolute maximum's frequency 
        n_approx = int(round(n)) # Round the midi number to fit a note
        return note_name(n_approx)

In [7]:
# Main 2
def main_2():
    hanning = 0.5 * (1 - np.cos(np.linspace(0,2*np.pi,WINDOW_SIZE_SAMPLES,False)))  # The hanning window function

    notes = []
    window_number = 0
    last_window = False
    while not(last_window):
        last_window, window = extract_window(signal, window_number)
        window_number += 1

        fft = np.fft.rfft(window * hanning)
        notes.append(detect_note_2(fft,window_number))
        
    print(notes)

In [13]:
main_2()

['C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'D#0', 'D#0', 'D#0', 'D4', 'C#4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D5', 'D4', 'D4', 'D#0', 'D4', 'D4', 'D#4', 'F4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E5', 'E5', 'E5', 'A1', 'E5', 'A1', 'E5', 'D#0', 'D#0', 'D#0', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F5', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F#4', 'G4', 'G4', 'G5', 'G4', 'G4', 'G4', 'G5', 'G4', 'G4', 'G4', 'G5', 'G4', 'A1', 'G5', 'D#0', 'G4', 'G4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'D#0', 'D#0', 'D#0', 'D#0', 'D#0', 'D#0', 'D#0', 'D#0', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'A1', 'B4', 'B4', 'D#0', 'D#0', 'D#0', 'D#0', 'D#0', 'D#0', 'D#0', 'D#0', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C6', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'A1', 'D#0', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'A1', 'B4', 'B4', 'B4', 'B4', 'A1', 'B1', 'B4', 'D#0', 'D#0', 'D#0', 'G#4', 'A4', '

Tercera aproximación: Hallar la fundamental en cada ventana como la frecuencia con mayor altura media en el conjunto de sus armónicos.

In [8]:
# Functions 3    
def fund_freq(freqs,F):                                                   # Returns estimate fundamental frequency
    means = np.array([])
    
    for i in range(1,len(F)):
        if F[i] <= INTENSITY_THRESHOLD:
            means = np.append(means,0)
        else:
            nharmonics = 1
            mean = 0
            harmonic = nharmonics * freqs[i]
            top_freq = (len(F)-1) * freqs[1] 
            while harmonic <= top_freq:
                h_pos = (np.abs(freqs - harmonic)).argmin()
                mean += F[h_pos]
                nharmonics += 1
                harmonic = nharmonics * freqs[i]
            
            mean = mean / nharmonics
            means = np.append(means,mean)
                
    return freqs[means.argmax()+1]
    
def detect_note_3(fft,wnum):                                                       # Returns the most suitable note for a given fft
    freqs = np.fft.rfftfreq(WINDOW_SIZE_SAMPLES, 1/SAMPLE_RATE) # The array of frequencies to evaluate in the fft
    F = np.abs(fft.real) # Evaluations of those frequencies
    graph2D(freqs[0:500],F[0:500],"Plots\\w" + str(wnum) + "_V3.pdf")
    
    if np.max(F) < INTENSITY_THRESHOLD: # If there are not high enough maximums, we cannot know
        return "Unknown"
    else:
        n = freq_to_number(fund_freq(freqs,F)) # Midi number of absolute maximum's frequency 
        n_approx = int(round(n)) # Round the midi number to fit a note
        return note_name(n_approx)

In [9]:
# Main 3
def main_3():
    hanning = 0.5 * (1 - np.cos(np.linspace(0,2*np.pi,WINDOW_SIZE_SAMPLES,False)))  # The hanning window function

    notes = []
    window_number = 0
    last_window = False
    while not(last_window):
        last_window, window = extract_window(signal, window_number)
        window_number += 1

        fft = np.fft.rfft(window * hanning)
        notes.append(detect_note_3(fft,window_number))
        
    print(notes)

In [13]:
main_3()

['C4', 'C4', 'C4', 'C4', 'C5', 'E6', 'C5', 'C5', 'G5', 'C5', 'C5', 'A#6', 'C5', 'A#6', 'C5', 'G5', 'A#6', 'D4', 'F#6', 'D4', 'D4', 'F#6', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D5', 'D4', 'D4', 'D5', 'D4', 'E5', 'E6', 'E5', 'E6', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'E5', 'B6', 'E5', 'B6', 'E7', 'F4', 'F4', 'F4', 'F4', 'F4', 'F5', 'F5', 'F5', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'D6', 'D7', 'G4', 'G5', 'D6', 'G5', 'D7', 'G5', 'D7', 'G5', 'G5', 'G5', 'G5', 'G5', 'G5', 'D7', 'G5', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A5', 'A5', 'A4', 'A4', 'A4', 'A4', 'B4', 'B4', 'B4', 'B4', 'B5', 'B4', 'B4', 'B6', 'B4', 'F#6', 'B4', 'B4', 'B5', 'B4', 'B4', 'B5', 'B4', 'B4', 'B5', 'C6', 'G6', 'C5', 'G6', 'C5', 'G6', 'C5', 'G6', 'C5', 'G6', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'B4', 'B4', 'B4', 'B4', 'F#7', 'B4', 'B4', 'B6', 'B4', 'B4', 'B4', 'B4', 'B5', 'B4', 'B4', 'F#6', 'B4', 'B4', 'A4', 'A4', 'A4', 'A4', 'A5', 'A5', '

Cuarta aproximación: Hallar la fundamental en cada ventana como la frecuencia con mayor altura media en sus primeros armónicos.

In [10]:
# Inputs
NUM_FIRST_HARMONICS = 5

In [11]:
# Functions 4
def fund_freq_2(freqs,F):                                                   # Returns estimate fundamental frequency
    means = np.array([])
    
    for i in range(1,len(F)):
        if F[i] <= INTENSITY_THRESHOLD:
            means = np.append(means,0)
        else:
            nharmonics = 1
            mean = 0
            harmonic = nharmonics * freqs[i]
            top_freq = (len(F)-1) * freqs[1] 
            while harmonic <= top_freq and nharmonics <= NUM_FIRST_HARMONICS:
                h_pos = (np.abs(freqs - harmonic)).argmin()
                mean += F[h_pos]
                nharmonics += 1
                harmonic = nharmonics * freqs[i]
            
            mean = mean / nharmonics
            means = np.append(means,mean)

    return freqs[means.argmax()+1]
    
def detect_note_4(fft,wnum):                                                       # Returns the most suitable note for a given fft
    freqs = np.fft.rfftfreq(WINDOW_SIZE_SAMPLES, 1/SAMPLE_RATE) # The array of frequencies to evaluate in the fft
    F = np.abs(fft.real) # Evaluations of those frequencies
    #graph2D(freqs[0:500],F[0:500],"Plots\\w" + str(wnum) + "_V4")
    
    if np.max(F) < INTENSITY_THRESHOLD: # If there are not high enough maximums, we cannot know
        return "Unknown"
    else:
        n = freq_to_number(fund_freq_2(freqs,F)) # Midi number of absolute maximum's frequency 
        n_approx = int(round(n)) # Round the midi number to fit a note
        return note_name(n_approx)

In [12]:
# Main 4
def main_4():
    hanning = 0.5 * (1 - np.cos(np.linspace(0,2*np.pi,WINDOW_SIZE_SAMPLES,False)))  # The hanning window function

    notes = []
    window_number = 0
    last_window = False
    while not(last_window):
        last_window, window = extract_window(signal, window_number)
        window_number += 1

        fft = np.fft.rfft(window * hanning)
        notes.append(detect_note_4(fft,window_number))
        
    print(notes)

In [16]:
main_4()

['C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C5', 'C4', 'C4', 'C4', 'C4', 'C5', 'C4', 'C4', 'C3', 'D#-1', 'D3', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'B0', 'D#0', 'D#-1', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'G#4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G5', 'G4', 'G4', 'G4', 'G5', 'G4', 'G4', 'G4', 'D#0', 'G4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'D#-1', 'D#-1', 'D#-1', 'D#-1', 'D#-1', 'D#-1', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'D#-1', 'D#-1', 'D#-1', 'D#-1', 'D#-1', 'D#-1', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'D#-1', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'D#0', 'D#-1', 'D#-1', 'A4', '

Quinta aproximación: Hallar la fundamental, de entre las frecuencias del sistema temperado, como aquella de mayor altura media en el conjunto de sus armónicos (siendo la altura de una frecuencia no conocida la media ponderada por la distancia de las alturas de las dos frecuencias conocidas más cercanas a ella).

In [13]:
# Global variables
MIN_MIDI_NUM = 21
MAX_MIDI_NUM = 108

In [19]:
# Functions 5
def indexes(freqs,i1,i2,harmonic):                                         # Returns h1 and h2 indexes of the nearest two 
    if i2-i1 == 1:                                                         #     harmonics of window's fund. to harmonic or
        if harmonic == freqs[i1]:                                          #     h1 the index of harmonic in freqs and h2<0
            return i1,-1
        elif harmonic == freqs[i2]:
            return i2,-1
        else:
            return i1,i2
    else:
        isplit = int(i1 + np.ceil((i2-i1)/2.0))
        if harmonic < freqs[isplit]:
            return indexes(freqs,i1,isplit,harmonic)
        elif harmonic > freqs[isplit]:
            return indexes(freqs,isplit,i2,harmonic)
        else:
            return isplit,-1

def fund_freq_3(freqs,F):                                                  # Returns estimate fundamental frequency
    means = np.array([])
    
    for i in range(MIN_MIDI_NUM,MAX_MIDI_NUM+1):
        fund = number_to_freq(i)
        nharmonics = 1
        mean = 0
        harmonic = nharmonics * fund
        top_freq = (len(F)-1) * freqs[1] 
        while harmonic <= top_freq:
            # Compute the indexes of the nearest two harmonics of window's fund. to harmonic
            h1,h2 = indexes(freqs,0,len(freqs)-1,harmonic) 
            if h2 < 0:
                if harmonic == fund and F[h1] <= INTENSITY_THRESHOLD: # Don't compute for powerless fundamentals
                    mean = 0
                    break
                    
                mean += F[h1]
            else:
                # Weighted mean of F[h1] and F[h2] by distance of freqs[h1] and freqs[h2] to the harmonic
                mean += (F[h1]*np.log2(freqs[h2]/harmonic) + F[h2]*np.log2(harmonic/freqs[h1])) / np.log2(freqs[h2]/freqs[h1]) 
                     
            nharmonics += 1
            harmonic = nharmonics * fund

        mean = mean / np.power(nharmonics,0.5)
        means = np.append(means,mean)

    return means.argmax()+MIN_MIDI_NUM
    
def detect_note_5(fft,wnum):                                               # Returns the most suitable note for a given fft
    freqs = np.fft.rfftfreq(WINDOW_SIZE_SAMPLES, 1/SAMPLE_RATE) # The array of frequencies to evaluate in the fft
    F = np.abs(fft.real) # Evaluations of those frequencies
    graph2D(freqs[0:500],F[0:500],"Plots\\w" + str(wnum) + "_V5.pdf")
    
    if np.max(F) < INTENSITY_THRESHOLD: # If there are not high enough maximums, we cannot know
        return "Unknown"
    else:
        n = fund_freq_3(freqs,F) # Midi number of absolute maximum's frequency
        return note_name(n)

In [15]:
# Main 5
def main_5():
    hanning = 0.5 * (1 - np.cos(np.linspace(0,2*np.pi,WINDOW_SIZE_SAMPLES,False)))  # The hanning window function

    notes = []
    window_number = 0
    last_window = False
    while not(last_window):
        last_window, window = extract_window(signal,window_number)
        window_number += 1

        fft = np.fft.rfft(window * hanning)
        notes.append(detect_note_5(fft,window_number))
        
    print(notes)

In [42]:
main_5()

['C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'A#0', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D5', 'D4', 'D4', 'D4', 'D4', 'C1', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E5', 'E5', 'E4', 'A0', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F5', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'B0', 'G4', 'G4', 'G5', 'G4', 'G4', 'G4', 'G5', 'G4', 'G4', 'G4', 'G5', 'G4', 'G4', 'G4', 'G4', 'G4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A5', 'A4', 'A4', 'A4', 'A4', 'A4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B5', 'B4', 'B4', 'B4', 'B4', 'B4', 'A0', 'A0', 'D1', 'C5', 'C5', 'C5', 'C5', 'C5', 'C6', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'A1', 'B4', 'B4', 'A#0', 'B0', 'A4', 'A4', 'A4', 'A4', 'A5', '

Sexta aproximación: Hallar la fundamental, de entre las frecuencias del sistema temperado, como aquella de mayor altura media en el conjunto de sus primeros armónicos (siendo la altura de una frecuencia no conocida la media ponderada por la distancia de las alturas de las dos frecuencias conocidas más cercanas a ella).

In [16]:
# Inputs
NUM_FIRST_HARMONICS = 5
# Global variables
MIN_MIDI_NUM = 21
MAX_MIDI_NUM = 108

In [17]:
# Functions 6
def fund_freq_4(freqs,F,wnum):                                                  # Returns estimate fundamental frequency
    means = np.array([])
    
    for i in range(MIN_MIDI_NUM,MAX_MIDI_NUM+1):
        fund = number_to_freq(i)
        nharmonics = 1
        mean = 0
        harmonic = nharmonics * fund
        top_freq = (len(F)-1) * freqs[1] 
        while harmonic <= top_freq and nharmonics < NUM_FIRST_HARMONICS:
            # Compute the indexes of the nearest two harmonics of window's fund. to harmonic
            h1,h2 = indexes(freqs,0,len(freqs)-1,harmonic) 
            if h2 < 0:
                if harmonic == fund and F[h1] <= INTENSITY_THRESHOLD: # Don't compute for powerless fundamentals
                    mean = 0
                    break
                    
                mean += F[h1]
            else:
                # Weighted mean of F[h1] and F[h2] by distance of freqs[h1] and freqs[h2] to the harmonic
                mean += (F[h1]*np.log2(freqs[h2]/harmonic) + F[h2]*np.log2(harmonic/freqs[h1])) / np.log2(freqs[h2]/freqs[h1])
                    
            nharmonics += 1
            harmonic = nharmonics * fund

        mean = mean / np.power(nharmonics,0.5)
        means = np.append(means,mean)

    return means.argmax()+MIN_MIDI_NUM
    
def detect_note_6(fft,wnum):                                               # Returns the most suitable note for a given fft
    freqs = np.fft.rfftfreq(WINDOW_SIZE_SAMPLES, 1/SAMPLE_RATE) # The array of frequencies to evaluate in the fft
    F = np.power(np.abs(fft.real),2) # Evaluations of those frequencies
    #graph2D(freqs[0:500],F[0:500],"Plots\\w" + str(wnum) + "_V6")
    
    if np.max(F) < INTENSITY_THRESHOLD: # If there are not high enough maximums, we cannot know
        return "Unknown"
    else:
        n = fund_freq_4(freqs,F,wnum) # Midi number of absolute maximum's frequency 
        return note_name(n)

In [18]:
# Main 6
def main_6():
    hanning = 0.5 * (1 - np.cos(np.linspace(0,2*np.pi,WINDOW_SIZE_SAMPLES,False)))  # The hanning window function

    notes = []
    window_number = 0
    last_window = False
    while not(last_window):
        last_window, window = extract_window(signal,window_number)
        window_number += 1

        fft = np.fft.rfft(window * hanning)
        notes.append(detect_note_6(fft,window_number))
        
    print(notes)

In [54]:
main_6()

['C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'D4', 'D4', 'D4', 'D4', 'D4', 'G2', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'G2', 'D4', 'D4', 'D3', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'A#2', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'C#3', 'G4', 'G4', 'C4', 'G4', 'G4', 'G4', 'C4', 'G4', 'G4', 'G4', 'C4', 'G4', 'G4', 'C4', 'G4', 'G4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A0', 'A4', 'A4', 'E3', 'E3', 'B4', 'E3', 'B4', 'B4', 'B4', 'E3', 'E3', 'B4', 'B4', 'B4', 'E4', 'B4', 'B4', 'A0', 'A0', 'A0', 'A0', 'C5', 'C5', 'F3', 'F3', 'F3', 'C4', 'F3', 'F4', 'F3', 'C3', 'F3', 'F3', 'F3', 'F3', 'F3', 'F3', 'F3', 'F3', 'E3', 'E3', 'B4', 'E3', 'B4', 'E3', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'A0', 'B4', 'B4', 'A0', 'B4', 'B4', 'G#3', 'D3', 'A4', 'A4', 'A4', 'A4', 'A4', 

Séptima aproximación: Autocorrelación.

In [23]:
# Functions 7
def ac1(window):
    ac = []
    s = 0
    for i in range(0,len(window)):
        for j in range(0,len(window)):
            s += window[j] * window[(j+i)%len(window)]
        ac.append(s)
        s = 0
    return ac

def ac2(window):
    ac = np.correlate(window,window,mode='full')
    return ac[int(len(ac)/2):]

def autocorrelation(window): # Fastest
    ac = correlate(window,window,mode='full')
    return ac[int(len(ac)/2):]

In [24]:
# Main 7
def main_7():
    hanning = 0.5 * (1 - np.cos(np.linspace(0,2*np.pi,WINDOW_SIZE_SAMPLES,False)))  # The hanning window function

    notes = []
    window_number = 0
    last_window = False
    while not(last_window):
        last_window, window = extract_window(signal,window_number)
        window_number += 1
        fft = np.fft.rfft(autocorrelation(window * hanning))
        notes.append(detect_note_5(fft,window_number))
        
    print(notes)

In [27]:
main_7()

['C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E5', 'E5', 'E5', 'E5', 'E5', 'E4', 'E4', 'E4', 'E4', 'E4', 'A0', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'A0', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G5', 'G5', 'G5', 'G5', 'G5', 'G4', 'G4', 'G4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'A0', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'A1', 'B4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4

Octava aproximación: Encontrar los picos relevantes (ignorar la intensidad de los picos) y estudiar qué frecuencia tiene mayor número de armónicos en dicho conjunto de picos.

In [24]:
# Functions 8
def remove_duplicates(seq): # Remove duplicates preserving order
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

def detect_peaks(freqs,F): # Returns the array of freqs where fft has relevant peaks
    pindex = find_peaks(F)
    kmeans = KMeans(n_clusters=2,n_init=5,random_state=123456)
    P = pd.DataFrame(data=[F[i] for i in pindex[0]],index=pindex[0],columns=['Intensity'])
    clusters = kmeans.fit_predict(P) # Detect two clusters: peaks and non peaks
    cluster_id = clusters[np.where(pindex[0] == np.argmax(F))[0][0]] # Cluster of peaks id
    rpindex = np.where(clusters == cluster_id)[0] # Indexes of relevant peaks
    return [freqs[i] for i in [pindex[0][j] for j in rpindex]]

def find_candidates(cset): # Finds candidates for fundamental by substracting elements in cset
    aux_cset = [c for c in cset if c >= 27.5] # Remove too low freqs
    aux_cset.sort() # Order
    aux_cset.insert(0,0) # Add 0 at the beginning
    candidates = []
    for i in range(0,len(aux_cset)):
        for j in range(i+1,len(aux_cset)):
            candidate = number_to_freq(int(round(freq_to_number(aux_cset[j] - aux_cset[i])))) # Round to equal temperament
            if candidate not in candidates:
                candidates.append(candidate)
    return [c for c in candidates if c >= 27.5] # Remove too low freqs

def count_harmonics(candidates): # Count the number of harmonics in candidates for each candidate
    aux_candidates = candidates.copy()
    aux_candidates.sort() # Order
    nharmonics = np.zeros(len(aux_candidates),dtype=int)
    for i in range(0,len(aux_candidates)):
        for j in range(i,len(aux_candidates)):
            div = np.modf(aux_candidates[j]/aux_candidates[i])[0]
            if np.abs(div-round(div)) < 0.01: # Check if harmonic
                nharmonics[i] += 1
    return nharmonics

def count_harmonics2(candidates,m): # Count the number of harmonics in candidates for each candidate
    aux_candidates = candidates.copy()
    aux_candidates.sort() # Order
    nharmonics = np.zeros(len(aux_candidates),dtype=int)
    for i in range(0,len(aux_candidates)):
        for j in range(i,len(aux_candidates)):
            div = np.modf(aux_candidates[j]/aux_candidates[i])[0]
            if np.abs(div-round(div)) < 0.01: # Check if harmonic
                nharmonics[i] += 1
    for i in range(0,len(aux_candidates)):
        samples = min(1,round(m/aux_candidates[i]))
        nharmonics[i] /= np.sqrt(samples)
    return nharmonics
    
def detect_note_7(fft,wnum):
    freqs = np.fft.rfftfreq(WINDOW_SIZE_SAMPLES, 1/SAMPLE_RATE) # The array of frequencies to evaluate in the fft
    F = np.abs(fft.real) # Evaluations of those frequencies
    #graph2D(freqs[0:500],F[0:500],"Plots\\w" + str(wnum) + "_V8")
    
    peaks = [number_to_freq(round(freq_to_number(i))) for i in detect_peaks(freqs,F)] # Round to equal temperament
    peaks = remove_duplicates(peaks) # Remove duplicates
    
    candidates = find_candidates(peaks)  
    nharmonics = count_harmonics2(candidates,freqs[-1])
    
    candidates.sort()
    
    return note_name(int(round(freq_to_number(candidates[np.argmax(nharmonics)])))) if candidates else "Unknown"

In [25]:
# Main 8
def main_8():
    hanning = 0.5 * (1 - np.cos(np.linspace(0,2*np.pi,WINDOW_SIZE_SAMPLES,False)))  # The hanning window function

    notes = []
    window_number = 0
    last_window = False
    while not(last_window):
        last_window, window = extract_window(signal,window_number)
        window_number += 1
        #print("Ventana " + str(window_number))
        fft = np.fft.rfft(window * hanning)
        notes.append(detect_note_7(fft,window_number))
        
    print(notes)

In [26]:
main_8()

['C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C1', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'E4', 'D#1', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E5', 'D5', 'E5', 'E4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F1', 'G4', 'G4', 'G5', 'G4', 'G4', 'G4', 'G5', 'G4', 'G4', 'G4', 'G5', 'G4', 'A1', 'G5', 'C3', 'A1', 'G1', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A1', 'A1', 'A5', 'A5', 'A1', 'A1', 'A1', 'A4', 'A#0', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'A1', 'B4', 'B4', 'A1', 'B5', 'A1', 'A1', 'A1', 'A#0', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C3', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'A1', 'B1', 'A#0', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B1', 'E2', 'B4', 'B1', 'A1', 'G#4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A5'

Novena aproximación: Mejora de la anterior. K-Means iterado sobre k hasta antes de superar un umbral de picos relevantes. Posteriormente, se aplica un proceso de corrección de notas teniendo en cuenta las anteriores a cada cual. Se añade un mecanismo de detección de silencio.

In [66]:
# Inputs
MAX_REL_PEAKS = 12 # Maximum number of peaks in the cluster of relevant peaks
N_NOTES_CORRECTION_L = 4 # Number of notes to the left to consider for correcting a note
N_NOTES_CORRECTION_R = 0 # Number of notes to the right to consider for correcting a note
MAX_KM_ITERATIONS = 50 # Maximum number of K-Means iterations (> 0)
MAX_CORRECTIONS = 1 # Maximum number of corrections
SILENCE_THRESHOLD = 0.0001 # Intensity threshold for silence in [0,1]

In [67]:
# Functions 9  
def remove_duplicates(seq): # Remove duplicates preserving order
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

def detect_peaks2(freqs,F): # Returns the array of freqs where fft has relevant peaks
    peaks_before = []
    peaks_after = []
    pindex = find_peaks(F)
    num_clusters = 2
    nIterations = 0
    P = pd.DataFrame(data=[F[i] for i in pindex[0]],index=pindex[0],columns=['Intensity'])
    
    kmeans = KMeans(n_clusters=num_clusters,n_init=5,random_state=123456)
    clusters = kmeans.fit_predict(P) # Detect two clusters: peaks and non peaks
    cluster_id = clusters[np.argmin([F[i] for i in pindex[0]])] # Cluster of non relevant peaks id
    rpindex = np.where(clusters != cluster_id)[0] # Indexes of relevant peaks
    peaks_after = [freqs[i] for i in [pindex[0][j] for j in rpindex]]
    num_clusters += 1
    peaks_before = peaks_after.copy()
    while len(peaks_after) <= MAX_REL_PEAKS and nIterations < MAX_KM_ITERATIONS:
        peaks_before = peaks_after.copy()
        nIterations += 1
        kmeans = KMeans(n_clusters=num_clusters,n_init=5,random_state=123456)
        clusters = kmeans.fit_predict(P) # Detect <num_clusters> clusters
        cluster_id = clusters[np.argmin([F[i] for i in pindex[0]])] # Cluster of non relevant peaks id
        rpindex = np.where(clusters != cluster_id)[0] # Indexes of relevant peaks
        peaks_after = [freqs[i] for i in [pindex[0][j] for j in rpindex]]
        num_clusters += 1
        
    return peaks_before

def find_candidates2(cset): # Finds candidates for fundamental by substracting elements in cset
    aux_cset = [c for c in cset if c >= 27.5].copy() # Remove too low freqs
    aux_cset.sort() # Order
    candidates = aux_cset.copy()
    for i in range(0,len(aux_cset)-1):
        candidate = number_to_freq(int(round(freq_to_number(aux_cset[i+1] - aux_cset[i])))) # Round to equal temperament
        if candidate not in candidates:
            candidates.append(candidate)
    return [c for c in candidates if c >= 27.5] # Remove too low freqs

def count_harmonics(candidates): # Count the number of harmonics in candidates for each candidate
    aux_candidates = candidates.copy()
    aux_candidates.sort() # Order
    nharmonics = np.zeros(len(aux_candidates),dtype=int)
    for i in range(0,len(aux_candidates)):
        for j in range(i,len(aux_candidates)):
            div = np.modf(aux_candidates[j]/aux_candidates[i])[0]
            if np.abs(div-round(div)) < 0.01: # Check if harmonic
                nharmonics[i] += 1
    return nharmonics

def count_harmonics2(candidates,m): # Count the number of harmonics in candidates for each candidate
    aux_candidates = candidates.copy()
    aux_candidates.sort() # Order
    nharmonics = np.zeros(len(aux_candidates),dtype=float)
    for i in range(0,len(aux_candidates)):
        for j in range(i,len(aux_candidates)):
            div = np.modf(aux_candidates[j]/aux_candidates[i])[0]
            if np.abs(div-round(div)) < 0.01: # Check if harmonic
                nharmonics[i] += 1
    for i in range(0,len(aux_candidates)):
        samples = max(1,round(m/aux_candidates[i]))
        nharmonics[i] /= np.power(samples,0.1)
        
    return nharmonics

def count_harmonics3(peaks,candidates,m): # Count the number of harmonics in peaks for each candidate
    nharmonics = np.zeros(len(candidates),dtype=float)
    for i in range(0,len(candidates)):
        for j in range(0,len(peaks)):
            if peaks[j] >= candidates[i]:
                div = np.modf(peaks[j]/candidates[i])[0]
                if np.abs(div-round(div)) < 0.01: # Check if harmonic
                    nharmonics[i] += 1
    for i in range(0,len(candidates)):
        samples = max(1,round(m/candidates[i]))
        nharmonics[i] /= np.power(samples,0.1)
        
    return nharmonics

def max_amplitude(fund,freqs,F): # Compute the maximum of amplitudes in fund harmonics as weight of the note
    max_amp = 0
    num_harmonic = 1
    harmonic = num_harmonic * fund
    top_freq = (len(F)-1) * freqs[1] 
    while harmonic <= top_freq:
        # Compute the indexes of the nearest two harmonics of window's fund. to harmonic
        h1,h2 = indexes(freqs,0,len(freqs)-1,harmonic) 
        if h2 < 0:
            max_amp = max([max_amp,F[h1]])
        else:
            # Weighted mean of F[h1] and F[h2] by distance of freqs[h1] and freqs[h2] to the harmonic
            max_amp = max([max_amp,(F[h1]*np.log2(freqs[h2]/harmonic) + F[h2]*np.log2(harmonic/freqs[h1])) / np.log2(freqs[h2]/freqs[h1])]) 

        num_harmonic += 1
        harmonic = num_harmonic * fund
        
    return max_amp
    
def detect_note_8(fft,wnum):
    freqs = np.fft.rfftfreq(WINDOW_SIZE_SAMPLES, 1/SAMPLE_RATE) # The array of frequencies to evaluate in the fft
    F = np.abs(fft.real) # Evaluations of those frequencies
    #graph2D(freqs[0:500],F[0:500],"Plots\\w" + str(wnum) + "_V9")
    
    peaks = [number_to_freq(round(freq_to_number(i))) for i in detect_peaks2(freqs,F)] # Round to equal temperament
    peaks = remove_duplicates(peaks) # Remove duplicates
     
    candidates = find_candidates2(peaks)
    nharmonics = count_harmonics3(peaks,candidates,freqs[-1]) 
    
    pred_fund = candidates[np.argmax(nharmonics)]

    return (note_name(int(round(freq_to_number(pred_fund)))),max_amplitude(pred_fund,freqs,F))

def correct_notes_iteration(notes,weights,nl,nr): # Correct each note according to its nl previous and nr following ones' weights
    cnotes = notes.copy() # Necessary for keeping the first and last notes
    cweights = weights.copy() # Necessary for keeping the first and last weights
    
    silence_threshold = SILENCE_THRESHOLD * max(cweights)
    n = max(nl,nr)
    w = [1] # New weights for the window, based on proximity to the note to be corrected
    for k in range(1,n+1):
        w.append(sum(w))
        
    for i in range(0,len(notes)-nr):
        if cnotes[i] == 'S': # Avoid correcting silence
            continue
        if cweights[i] <= silence_threshold: # Correct as silence and keep the weight
            cnotes[i] = 'S'
            continue
            
        if i in range(0,nl) or i in range(len(notes)-nr,len(notes)): # Skip if cannot be corrected for being out of the range
            continue
        
        nsubset = []
        wsubset = []
        nsums = []
        for j in range(i-nl,i+nr+1):
            if notes[j] not in nsubset:
                nsubset.append(notes[j])
                if j <= i:
                    wsubset.append(w[j-(i-n)] * weights[j])
                    nsums.append(w[j-(i-n)])
                else:
                    wsubset.append(w[i+n-j] * weights[j] / 2) # Little penalization to future notes
                    nsums.append(w[i+n-j] / 2)
            else:
                if j <= i:
                    wsubset[nsubset.index(notes[j])] += w[j-(i-n)] * weights[j]
                    nsums[nsubset.index(notes[j])] += w[j-(i-n)]
                else:
                    wsubset[nsubset.index(notes[j])] += w[i+n-j] * weights[j] / 2
                    nsums[nsubset.index(notes[j])] += w[i+n-j] / 2
                  
        index = len(wsubset) - wsubset[::-1].index(max(wsubset)) - 1 # Index of last maximum of wsubset
        cnotes[i] = nsubset[index]
        cweights[i] = wsubset[index] / nsums[index]
        
    return cnotes,cweights

def correct_notes(notes,weights,nl,nr): # Correct the notes
    count = 0
    
    notes_before = notes.copy()
    notes_after,cweights = correct_notes_iteration(notes,weights,nl,nr)
    while not np.array_equal(notes_before,notes_after) and count < MAX_CORRECTIONS:
        notes_before = notes_after.copy()
        count += 1
        notes_after,cweights = correct_notes_iteration(notes_before,cweights,nl,nr)
    
    return notes_before

In [68]:
# Main 9
def main_9():
    hanning = 0.5 * (1 - np.cos(np.linspace(0,2*np.pi,WINDOW_SIZE_SAMPLES,False)))  # The hanning window function

    notes = []
    weights = []
    window_number = 0
    last_window = False
    while not(last_window):
        last_window, window = extract_window(signal,window_number)
        window_number += 1
        #print("Ventana " + str(window_number))
        fft = np.fft.rfft(autocorrelation(window * hanning))
        note,weight = detect_note_8(fft,window_number)
        notes.append(note)
        weights.append(weight)
        
    cnotes = correct_notes(notes,weights,N_NOTES_CORRECTION_L,N_NOTES_CORRECTION_R)
        
    print(cnotes)
    #print([(cnotes[i],i+1) for i in range(0,len(cnotes))])

In [69]:
main_9()

['C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'C4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'D4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'E4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'F4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'G4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A1', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'C5', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'B4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4', 'A4

Por curiosidad: En lugar del número de armónicos de cada candidato en los picos, contabilizar las diferencias entre cada par de picos.

In [ ]:
# Inputs
PATH = 'C:\\Users\\Javier\\Desktop\\TFG\\ReconocimientoDeTonos'      # Path of project's directory
AUDIO_FILE = 'DoM-piano.wav'                                         # Audio file's name
WINDOW_SIZE_SECS = 0.2                                               # Size of the fft window in seconds
OVERLAPPING_SECS = 0.15                                              # Window's overlapping in seconds
SILENCE_THRESHOLD = 0.0001                                           # Intensity threshold for silence in [0,1]
INTENSITY_THRESHOLD = 0.001                                          # Intensity (relevance) threshold for frequencies
MAX_REL_PEAKS = 12                                                   # Maximum number of peaks in the cluster of relevant peaks
N_NOTES_CORRECTION_L = 4                                             # Number of notes to the left to consider for correcting a note
N_NOTES_CORRECTION_R = 0                                             # Number of notes to the right to consider for correcting a note
MAX_KM_ITERATIONS = 50                                               # Maximum number of K-Means iterations
MAX_CORRECTIONS = 1                                                  # Maximum number of corrections

# Global variables 
NOTES = ["C","C#","D","D#","E","F","F#","G","G#","A","A#","B"]       # The twelve notes' names
SAMPLE_RATE, data = wavfile.read(os.path.join(PATH,AUDIO_FILE))      # Get sample rate (samples per second) and signal data
signal = data if data.ndim == 1 else data.T[0]                       # Get the first channel
WINDOW_SIZE_SAMPLES = int(SAMPLE_RATE * WINDOW_SIZE_SECS)            # Size of the fft window in samples
OVERLAPPING_SAMPLES = int(SAMPLE_RATE * OVERLAPPING_SECS)            # Size of overlapping in samples
AUDIO_SIZE_SECS = len(signal) / SAMPLE_RATE                          # Size of the audio file in seconds

# Files' statistics
print("Sample rate: " + str(SAMPLE_RATE) + " Hz")                   
print("Signal: " + str(signal))                                      
print("Window size: " + str(WINDOW_SIZE_SECS) + " s = " + str(WINDOW_SIZE_SAMPLES) + " samples")
print("Overlapping: " + str(OVERLAPPING_SECS) + " s = " + str(OVERLAPPING_SAMPLES) + " samples")
print("Audio length: " + str(AUDIO_SIZE_SECS) + " s")

# Functions
def freq_to_number(f):                                                      # Transforms any note's frequency into its midi number 
    return 69 + 12*np.log2(f/440.0)    

def number_to_freq(n):                                                      # Transforms any note's midi number into its frequency
    return 440 * 2.0**((n-69)/12.0)

def note_name(n):                                                           # Gets the note's name given its midi number
    return NOTES[n % 12] + str(int(n/12 - 1))

def extract_window(audio, window_number):                                   # Returns samples of window number <window-number> and true or false whether it's the last window 
    begin = window_number * (WINDOW_SIZE_SAMPLES - OVERLAPPING_SAMPLES)
    end = begin + WINDOW_SIZE_SAMPLES
    
    if end < len(signal): # Commonly
        return False, audio[begin:end]
    else: # The window surpasses the audio data => Complete last elements of the window with zeros
        return True, np.concatenate([audio[begin:len(signal)-1],np.zeros(end-len(signal)+1,dtype=float)])
    
def autocorrelation(window):                                                # Autocorrelation of a given window
    ac = correlate(window,window,mode='full')
    return ac[int(len(ac)/2):]
    
def indexes(freqs,i1,i2,harmonic):                                          # Returns h1 and h2 indexes of the nearest two 
    if i2-i1 == 1:                                                          #     harmonics of window's fund. to harmonic or
        if harmonic == freqs[i1]:                                           #     h1 the index of harmonic in freqs and h2<0
            return i1,-1
        elif harmonic == freqs[i2]:
            return i2,-1
        else:
            return i1,i2
    else:
        isplit = int(i1 + np.ceil((i2-i1)/2.0))
        if harmonic < freqs[isplit]:
            return indexes(freqs,i1,isplit,harmonic)
        elif harmonic > freqs[isplit]:
            return indexes(freqs,isplit,i2,harmonic)
        else:
            return isplit,-1
        
def remove_duplicates(seq): # Remove duplicates preserving order
    seen = set()
    seen_add = seen.add
    return [x for x in seq if not (x in seen or seen_add(x))]

def detect_peaks(freqs,F): # Returns the array of freqs where fft has relevant peaks
    peaks_before = []
    peaks_after = []
    pindex = find_peaks(F)
    if not len(pindex[0]):
        return []
    num_clusters = 2
    nIterations = 0
    P = pd.DataFrame(data=[F[i] for i in pindex[0]],index=pindex[0],columns=['Intensity'])
    
    kmeans = KMeans(n_clusters=num_clusters,n_init=5,random_state=123456)
    clusters = kmeans.fit_predict(P) # Detect two clusters: peaks and non peaks
    cluster_id = clusters[np.argmin([F[i] for i in pindex[0]])] # Cluster of non relevant peaks id
    rpindex = np.where(clusters != cluster_id)[0] # Indexes of relevant peaks
    peaks_after = [freqs[i] for i in [pindex[0][j] for j in rpindex]]
    num_clusters += 1
    peaks_before = peaks_after.copy()
    while len(peaks_after) <= MAX_REL_PEAKS and nIterations < MAX_KM_ITERATIONS:
        peaks_before = peaks_after.copy()
        nIterations += 1
        kmeans = KMeans(n_clusters=num_clusters,n_init=5,random_state=123456)
        clusters = kmeans.fit_predict(P) # Detect <num_clusters> clusters
        cluster_id = clusters[np.argmin([F[i] for i in pindex[0]])] # Cluster of non relevant peaks id
        rpindex = np.where(clusters != cluster_id)[0] # Indexes of relevant peaks
        peaks_after = [freqs[i] for i in [pindex[0][j] for j in rpindex]]
        num_clusters += 1
        
    return peaks_before

def find_candidates(cset): # Finds candidates for fundamental by substracting elements in cset
    aux_cset = [c for c in cset if c >= 27.5].copy() # Remove too low freqs
    aux_cset.sort() # Order
    candidates = aux_cset.copy()
    for i in range(0,len(aux_cset)-1):
        candidate = number_to_freq(int(round(freq_to_number(aux_cset[i+1] - aux_cset[i])))) # Round to equal temperament
        if candidate not in candidates:
            candidates.append(candidate)
    return [c for c in candidates if c >= 27.5] # Remove too low freqs

def count_harmonics(peaks,candidates,m): # Count the number of harmonics in peaks for each candidate
    nharmonics = np.zeros(len(candidates),dtype=float)
    for i in range(0,len(candidates)):
        for j in range(0,len(peaks)):
            if peaks[j] >= candidates[i]:
                div = np.modf(peaks[j]/candidates[i])[0]
                if np.abs(div-round(div)) < 0.01: # Check if harmonic
                    nharmonics[i] += 1
    for i in range(0,len(candidates)):
        samples = max(1,round(m/candidates[i]))
        nharmonics[i] /= np.power(samples,0.1)
        
    return nharmonics

def count_differences(peaks): # Count the number of harmonics in peaks for each candidate
    aux_peaks = peaks.copy()
    aux_peaks.sort()
    aux_peaks.insert(0,0.0);
    differences = []
    counts = []
    
    for i in range(0,len(aux_peaks)):
        for j in range(i+1,len(aux_peaks)):
            diff = number_to_freq(int(round(freq_to_number(aux_peaks[j] - aux_peaks[i]))))
            if diff not in differences:
                differences.append(diff)
                counts.append(1)
            else:
                counts[differences.index(diff)] += 1
        
    return differences[np.argmax(counts)]

def max_amplitude(fund,freqs,F): # Compute the maximum of amplitudes in fund harmonics as weight of the note
    max_amp = 0
    num_harmonic = 1
    harmonic = num_harmonic * fund
    top_freq = (len(F)-1) * freqs[1] 
    while harmonic <= top_freq:
        # Compute the indexes of the nearest two harmonics of window's fund. to harmonic
        h1,h2 = indexes(freqs,0,len(freqs)-1,harmonic) 
        if h2 < 0:
            max_amp = max([max_amp,F[h1]])
        else:
            # Weighted mean of F[h1] and F[h2] by distance of freqs[h1] and freqs[h2] to the harmonic
            if freqs[h1] != 0:
                max_amp = max([max_amp,(F[h1]*np.log2(freqs[h2]/harmonic) + F[h2]*np.log2(harmonic/freqs[h1])) / np.log2(freqs[h2]/freqs[h1])])

        num_harmonic += 1
        harmonic = num_harmonic * fund
        
    return max_amp
    
def detect_note(fft):
    freqs = np.fft.rfftfreq(WINDOW_SIZE_SAMPLES, 1/SAMPLE_RATE) # The array of frequencies to evaluate in the fft
    F = np.abs(fft.real) # Evaluations of those frequencies
    
    peaks = [number_to_freq(round(freq_to_number(i))) for i in detect_peaks(freqs,F)] # Round to equal temperament
    if not len(peaks):
        return ('S',0)
    peaks = remove_duplicates(peaks) # Remove duplicates
     
    pred_fund = count_differences(peaks)
    
    return (note_name(int(round(freq_to_number(pred_fund)))),max_amplitude(pred_fund,freqs,F))

def correct_notes_iteration(notes,weights,nl,nr): # Correct each note according to its nl previous and nr following ones' weights
    cnotes = notes.copy() # Necessary for keeping the first and last notes
    cweights = weights.copy() # Necessary for keeping the first and last weights
    
    silence_threshold = SILENCE_THRESHOLD * max(cweights)
    n = max(nl,nr)
    w = [1] # New weights for the window, based on proximity to the note to be corrected
    for k in range(1,n+1):
        w.append(sum(w))
        
    for i in range(0,len(notes)-nr):
        if cnotes[i] == 'S': # Avoid correcting silence
            continue
        if cweights[i] <= silence_threshold: # Correct as silence and keep the weight
            cnotes[i] = 'S'
            continue
            
        if i in range(0,nl) or i in range(len(notes)-nr,len(notes)): # Skip if cannot be corrected for being out of the range
            continue
        
        nsubset = []
        wsubset = []
        nsums = []
        for j in range(i-nl,i+nr+1):
            if notes[j] not in nsubset:
                nsubset.append(notes[j])
                if j <= i:
                    wsubset.append(w[j-(i-n)] * weights[j])
                    nsums.append(w[j-(i-n)])
                else:
                    wsubset.append(w[i+n-j] * weights[j] / 2) # Little penalization to future notes
                    nsums.append(w[i+n-j] / 2)
            else:
                if j <= i:
                    wsubset[nsubset.index(notes[j])] += w[j-(i-n)] * weights[j]
                    nsums[nsubset.index(notes[j])] += w[j-(i-n)]
                else:
                    wsubset[nsubset.index(notes[j])] += w[i+n-j] * weights[j] / 2
                    nsums[nsubset.index(notes[j])] += w[i+n-j] / 2
                  
        index = len(wsubset) - wsubset[::-1].index(max(wsubset)) - 1 # Index of last maximum of wsubset
        cnotes[i] = nsubset[index]
        cweights[i] = wsubset[index] / nsums[index]
        
    return cnotes,cweights

def correct_notes(notes,weights,nl,nr): # Correct the notes
    count = 0
    
    notes_before = notes.copy()
    notes_after,cweights = correct_notes_iteration(notes,weights,nl,nr)
    while not np.array_equal(notes_before,notes_after) and count < MAX_CORRECTIONS:
        notes_before = notes_after.copy()
        count += 1
        notes_after,cweights = correct_notes_iteration(notes_before,cweights,nl,nr)
    
    return notes_before

def notes_per_window():
    hanning = 0.5 * (1 - np.cos(np.linspace(0,2*np.pi,WINDOW_SIZE_SAMPLES,False)))  # The hanning window function

    notes = []
    weights = []
    window_number = 0
    last_window = False
    while not(last_window):
        last_window, window = extract_window(signal,window_number)
        window_number += 1
        fft = np.fft.rfft(autocorrelation(window * hanning))
        note,weight = detect_note(fft)
        notes.append(note)
        weights.append(weight)
       
    return correct_notes(notes,weights,N_NOTES_CORRECTION_L,N_NOTES_CORRECTION_R)    

In [ ]:
npw = notes_per_window()
print(npw)